In [1]:
import chromadb
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import torch

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Create Chromadb and initialize model for embeddings

In [ ]:
persistentClient = chromadb.PersistentClient(path="recipe_vector_db")
recipe_collection = persistentClient.get_or_create_collection( 
       name="recipes", 
       metadata={"hnsw:space": "cosine"}
)

model = SentenceTransformer("all-MiniLM-L6-v2")
model = model.to(device)

### Create Embeddings

In [ ]:
df = pd.read_csv("data/recipe_store.csv", delimiter=";")
df = df.drop(columns=['Unnamed: 0'])

def combine_recipe(row):
    return f"{row['title']} {row['ingredients']} {row['directions']}"

# Generate embeddings for each recipe
recipes = df.apply(combine_recipe, axis=1).tolist()
embeddings = model.encode(recipes, show_progress_bar=True)

In [ ]:
print(len(embeddings))

223115


### Create vector store

In [ ]:
for idx, recipe in tqdm(enumerate(recipes),total=len(recipes)):
    recipe_collection.add(
    documents=[recipe],  # the text for each recipe
    metadatas=[{"title": df.iloc[idx]["title"]}],  # metadata: title
    embeddings=[embeddings[idx]],  # corresponding embedding
    ids=str(idx)
    )

100%|██████████| 223115/223115 [1:07:14<00:00, 55.31it/s]


### Query the vector store

In [ ]:
query = "Potatoes"
query_embedding = model.encode([query])

results = recipe_collection.query(
    query_embeddings=query_embedding,
    n_results=5  
)

for result in results['documents'][0]:
    print(result)


Potato Soup potatoes, diced, onion, chopped Boil until potatoes are done, Mash with mixer.
Seasoned Potatoes potatoes, canola oil, seasoned salt Use amounts depending on servings needed.
Surprise Potatoes 9 medium potatoes, 1 1/2 pt. (24 ounces) half and half, salt and pepper, onion powder, butter Boil potatoes with skins on, Cool in refrigerator (overnight is best), Then pare cooled potatoes, Grate 1/2 of the potatoes into a buttered casserole dish (13 x 9 x 2-inch), Sprinkle with salt, pepper and onion powder, Dot with butter.
Smothered Potatoes 3 to 4 large potatoes, 1/2 cups minced onions, salt and pepper to taste Dice potatoes and along with onions brown quickly in small amount of grease in skillet, After browning, add enough water to cover, cover and simmer until tender, Add water if they cook down and are not quite soft enough, Serves 3 to 4 people.
Lemon-Parsleyed Potatoes 1 1/2 pound tiny new potatoes, 1/4 cups butter, 1/4 cups snipped parsley, 1 tablespoon lemon juice Wash an